In [ ]:
import sys
import os
import re
import copy
import numpy as np
import scipy as sp
from scipy.interpolate import interp1d
import math 
import random
import itertools
from typing import List, Optional 
from os import listdir
from os.path import isfile, join
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate, optimize
from tqdm.notebook import tqdm

# load the various FastSim libraries
import DetectorSimulation.Detector as Detector
import DetectorSimulation.Particle as Particle
import DetectorSimulation.Vertex as Vertex
import DetectorSimulation.lorentz_transformation as lt
import DetectorSimulation.llp_gun as llp_gun
## depending on which fvs are used load a different specialized llp gun (llp_gun_new) (built on top of llp_gun)
import DetectorSimulation.llp_gun_new as lg
## NOTE: the various hadronic functions in llp_gun_new must be updated with the correct path of the LLP hadronic decay 
## 4-vector files depending on the type of analysis 

## These hadronic functions are almost the same but were separated into different functions for ease of use with the 
## different LLP analyses - but should be very easy to update them for any new analysis that need be done

# load helper function library for later
from Helpers.functions import *


### IMPORTANT ###
# All the hadronic decay 4-vector files are in the rest frame of the parent LLP (see files) and the code is set-up to boost them 
# to lab frame and deal with the same

# To use hadronic decay 4-vector files not in the rest frame of the parent LLP, one needs to comment/uncomment certain lines 
# in the llp_gun.py file 

## We give a demo of some basic functionalities. We also give a first look at the reconstruction/trigger capabilities of FastSim

## Single Particle manually

We give an example of generating a single particle event and checking its reconstruction/trigger.

In [ ]:
## Single PARTICLE MANUALLY

# Let's start with some code taken from the get_weight function itself

## First, let us setup the detector by choosing a param_card
param_card = "param_card_CDR.txt"
# change this to the path to param_card on your local computer
path_to_param_card = join(os.getcwd(), param_card)

# setup detector
detector_benchmark = Detector.Detector(path_to_param_card)

# clear the detector of past events, if any
detector_benchmark.clear_detector()

# choose a position for the LLP to decay at (starting point is always interaction point (IP)) 
# lets take a point in MATHUSLA and the sim automatically uses kinematics to figure out trajectory
position = (0,120,70)

# create an LLP particle, Particle.Particle(position, 4-vector, PID)
# play around with various 4-vectors and positions!
four_p = (100, 0, 100, 60) # (E, px, py, pz)
pid = 13 # let's say it is a muon (try changing the pid to an inivisible particle, say PID = 12, electron-neutrino)
mass = 1 # GeV

particle = Particle.Particle(position, four_p, pid, mass)

# feed this particle to the detector as a new (particle) event - other options include new_vertex_event which 
# feeds a vertex (i.e. an LLP with its decay products) to the detector
detector_benchmark.new_particle_event(particle)

# show/save event display (if needed) using the following (MORE information in the next few blocks)
# setting argument show = True displays and saves
# show = False saves but does not display
filename = join(os.getcwd(), "hello_MATHUSLA")
detector_benchmark.detector_display(filename, show=True)

## CHECKING RECONSTRUCTION/TRIGGER FOR A SINGLE PARTICLE EVENT 

# note the criteria shown in bottom right of the detector display are for vertices only, since they are the most 
# important objects we deal with, but we can check criterion for single particle events as follows:

print("particle passed through 4 tracker layers:", detector_benchmark.track_reconstructed(recon_criteria="medium"))

print("particle passed trigger criteria:", detector_benchmark.event_pass_trigger())


# we can also directly get all the information about this particle after it passed through detector
# UNCOMMENT next 3 lines to use
# par = detector_benchmark.return_current_particle() 
# just print the object to get info
# print("Here is some information about the particle\n\n", par)

## LLP Vertex manually

We give an example of generating a vertex event. Checking for reconstruction/trigger is in the "LLP Vertex from file" block

In [ ]:
# LLP VERTEX MANUALLY

# same as before
param_card = "param_card_CDR.txt"
path_to_param_card = join(os.getcwd(), param_card)

detector_benchmark = Detector.Detector(path_to_param_card)
detector_benchmark.clear_detector()

position = (0,120,70)
four_p = (100, 0, 100, 60) # random 4-vector (might not equal sum of daughter 4-vectors, but we use it for demo anyway)
pid = 1023

# create daughter particles for the vertex 
daughter1 = Particle.Particle(position, (6, 0.2, -2.1, 3.7), 13)
daughter2 = Particle.Particle(position, (0.7, 0, 0.3, 0.4), 211)
daughter3 = Particle.Particle(position, (2, 0.6, 1, -0.2), 13)

decay_products = [daughter1, daughter2, daughter3]

# create the vertex
vertex = Vertex.Vertex(position, four_p, pid, decay_products)
detector_benchmark.new_vertex_event(vertex)

filename = join(os.getcwd(), "hello_MATHUSLA")
detector_benchmark.detector_display(filename, show=True)

# now lets get some information about this vertex after it passed through detector

# we can also directly get all the information about this particle after it passed through detector
# UNCOMMENT next 3 lines to use
# ver = detector_benchmark.return_current_vertex() 
# # just print the object to get info
# print(ver)

## LLP Vertex from file

We give an example of reading in an LLP vertex from file (could be hadronic, leptonic 2-body or 3-body) and checking its reconstruction/trigger/. 

We give an example of generating a leptonic 2-body or 3-body vertex directly using the FastSim in the "explicit demo" block of code.

In [ ]:
# LLP VERTEX FROM FILE

# same as before
param_card = "param_card_CDR.txt"
path_to_param_card = join(os.getcwd(), param_card)

detector_benchmark = Detector.Detector(path_to_param_card)

##################
# change this to the path to bb_15.txt on your local machine
# path_to_file = "/Users/jai/Desktop/MATHUSLA_FastSim/bb_15.txt"
path_to_file = join(os.getcwd(), "bb_15.txt")
initial_position = (0,0,0) # initial position of the vertex is IP

# get 1 vertex (in LLP rest frame)
vertex = llp_gun.create_llp_from_file(path_to_file, initial_position, num=1)[0] # indexing as function returns a list

# next, aim the LLP at some specific decay target position and boost LLP momentum to new momentum if specified as follows
target = (0,120,70) # re-input target position
p_norm = 100 # momentum LLP vertex (i.e. decay products) should be boosted to, we use the masses buit into 
#              vertex.decay_product to get the boost for each daughter
boosted_vertex = llp_gun.align_trajectory(vertex, target, p_norm)

# feed in vertex
detector_benchmark.clear_detector()
detector_benchmark.new_vertex_event(boosted_vertex)

filename = join(os.getcwd(), "hello_MATHUSLA")
print("Boosted LLP vertex\n")
detector_benchmark.detector_display(filename, show=True)

## if we look at the top right figure, it is clear that there are tracks passing through ALL the ceiling sensor planes,
## however the top middle figure does not show the same picture, in fact, it might seem as if the tracks do not pass
## through ALL the ceiling sensors -- THIS IS JUST AN ARTIFACT OF THE MATPLOTLIB LIBRARY NOT BEING ABLE TO PLOT 2D
## PROJECTIONS OF 3D OBJECTS "NICELY", AND AS SUCH OVERLAYS/INTERSECTIONS ARE NOT EXACT ENOUGH - in this figure,
## the detector planes always overlay the tracks

## ## both these views are provided as a sanity check for the user can be accessed by setting the argument zorder 
## in the detector_display function to zorder = 1000 (so zorder only takes TWO values, 10 (default) or 1000)

# UNCOMMENT to use
# detector_benchmark.detector_display(filename, show=True, zorder=1000)

## CHECKING RECONSTRUCTION/TRIGGER

# how to check if LLP vertex passed DVmedium2 (DV2, see paper) recon criteria
if detector_benchmark.vertex_reconstructed(recon_criteria="DVmedium2"): # gives 0 or 1 
    print("passed DV2")
else:
    print("did not pass DV2")

# how to check if LLP vertex passed DVmedium2 (DV2, see paper) recon criteria
if detector_benchmark.vertex_reconstructed(recon_criteria="DVmedium3"): # gives 0 or 1 
    print("passed DV3")
else:
    print("did not pass DV3")

# how to check if LLP vertex passed trigger (nearest_neighbour, see paper) criteria
# NOTE: we use detector_benchmark.event_pass_trigger(trigger_criteria) without any argument as there is only one criteria
# and the simulation automatically uses that as default
if detector_benchmark.event_pass_trigger():
    print("passed trigger")
else:
    print("did not pass trigger")
    
# Finally we can take a product of recon and trigger to find whether LLP vertex passed both
if detector_benchmark.vertex_reconstructed(recon_criteria="DVmedium2") * detector_benchmark.event_pass_trigger():
    print("passed both recon + trigger")
else:
    print("did not pass recon or trigger or maybe both")


## Explicit demo - in essence a mini-version of the pipeline used to generate our results

There are 3 lines that can be commented or uncommented to play with the type of decay - hadronic, leptonic2body or leptonic3body.

In [ ]:
# choose a mass for the LLP (in GeV)
m = 1.0

# say we read in ctau from a file
ctau = 100

# choose a param card - we choose the default CDR geometry as before
param_card = "param_card_CDR.txt"
path_to_param_card = join(os.getcwd(), param_card)

# let's say we are considering SM+S produced LLP decay to hadrons. We read in the possible decay products from the
# file "hadrons1to2gev_1.0.txt" and use them to construct vertices and shoot them at MATHUSLA

# first, read in say 20 (or however many you might want) LLP 4-vectors from "SMS_LLPweight4vectorBmesonlist_mS_1.0.csv"
fv_path = join(os.getcwd(), "SMS_LLPweight4vectorBmesonlist_mS_1.0.csv")

# use the read_vectors_SMS file to read in, as format of 4-vectors is (weight, E, px, py, pz) - we ignore the weight in this the demo
vectors = read_vectors_SMS(fv_path, 20) # 4-vectors from B parents

# note length of read-in 4-vector list as we use this to iterate through this list later
num_iter = len(vectors)

# finally run the simulation - setup the detector by giving it the path to the chosen param_card
# change path to local machine
detector_benchmark = Detector.Detector(path_to_param_card)

# get angle ranges for MATHUSLA and can compare LLP 4-vectors (with starting point being the IP)
# with the same to check if they pass through the detector decay volume
phi_min, phi_max, theta_min, theta_max = get_detector_angles(detector_benchmark)

################## actual simulation ################ 

# iterate through the LLP 4-vectors
for k in tqdm(range(num_iter)): 
    print("LLP",k,"\n")
    # forget about first entry (i.e. weight) and only keep track of actual 4-vector
    four_p = vectors[k][1:]
    # get the LLP theta and phi angles 
    llp_theta = get_theta(four_p[1:])
    llp_phi = get_phi(four_p[1:])
    # check if LLP passes theta (equivalently eta) range of MATHUSLA i.e. it has chance of passing through
    ## (NOTE: while theta check should ensure that LLP SHOULD pass through, sometimes it still might not
    #       due on numerical precision etc - all these sanity checks are built in at every stage)
    if (llp_theta > theta_min) and (llp_theta < theta_max):
        # if it does, randomly rotate the phi of the LLP 4-vector
        four_p_rot = deal_with_phi(four_p, phi_min, phi_max)
        # use new 4-vector, lifetime of LLP (say 100m), and the detector to generate
        # weight, decay position, boost of the LLP
        pack = get_weight(four_p_rot, m, ctau, detector_benchmark) # returns pack = (weight, decay_position, boost)           

        # if get_weight does not return None i.e. LLP enter and exits detector (and there is no numerical precision issue etc.)
        if pack is not None:
            print("LLP passed through MATHUSLA")
            # once again, clear the detector of previous events (if any)
            detector_benchmark.clear_detector()

#########################################################################################################

            # hadronic decay from file 
    
            ## UNCOMMENT TO USE, COMMENT OUT TO TEST OTHER DECAYS
#             llp = lg.get_llp("hadronic_SMS", m, pack[1], pack[2],"hadrons1to2gev")
            
            # or create leptonic 2-body decay to e+e- (can input the same pid for both i.e. 11 as the fastsim only sees charge, not the type of charge)
            
            ## UNCOMMENT TO USE, COMMENT OUT TO TEST OTHER DECAYS
#             llp = lg.get_llp("leptonic2body", m, pack[1], pack[2],[11,11])
            
            # or create leptonic 3-body decay to e+e-nu

            ## UNCOMMENT TO USE, COMMENT OUT TO TEST OTHER DECAYS
#             llp = lg.get_llp("leptonic3body", m, pack[1], pack[2],[11,11,12])

#########################################################################################################

            # make sure LLP is not None i.e. not an invisible decay
            if llp is not None:
                detector_benchmark.new_vertex_event(llp)
                
                filename = join(os.getcwd(), "hello_MATHUSLA")
                detector_benchmark.detector_display(filename, show=True)
                
                # the sim has run now, so we can check if LLP was reconstructed using different criteria
                
                print("LLP DV2 recon:", detector_benchmark.vertex_reconstructed(recon_criteria="DVmedium2"))
                print("LLP DV3 recon:", detector_benchmark.vertex_reconstructed(recon_criteria="DVmedium3"))
                print("LLP trigger:", detector_benchmark.event_pass_trigger(),"\n")
                
                
            else:
                print("Invisible decay\n")
                
        else:
            print("LLP did not pass through MATHUSLA\n")
    else:
        print("LLP did not pass through MATHUSLA\n")
        

## Bonus: Converting hadronic decay 4-vectors into Fastim format

While we have provided links to repositories containing the hadronic decay products required for various LLP analyses, the FastSim takes them as input in a specific format (for e.g. look at file "bb_15.txt" or "SMS_LLPweight4vectorBmesonlist_mS_1.0.csv"). Here we demo the script to convert the given files into such a format.

In [ ]:
from Helpers.script import *

# we take in a hadronic decay 4-vectors file similar to the ones at the github links in the paper and 
# convert it into the format needed for input into the FastSim

# first argument is where to save output, second argument is where to read in the file to be converted
# change paths as needed
write_hadrons_for_fastsim(join(os.getcwd(), "sample_fastsim_input.txt"), join(os.getcwd(), "to_be_converted_to_fastsim_input_format.txt"))